# Training the model with no shuffled dataset

In [1]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
# from fastai.column_data import *
# from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.torch_imports import save_model, load_model

In [4]:
from models import *
from utils import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
ls -f KITTI/ | grep .csv

validation.csv
validation_192_640.csv
training.csv
validation_192_640_pre.csv
training_192_640_pre.csv
training_192_640.csv


In [6]:
train, val = pd.read_csv('./KITTI/training_192_640_pre.csv'), pd.read_csv('./KITTI/validation_192_640_pre.csv')
val = shuffle(val)

In [7]:
train.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,370.156573,367.787571,309.379246,92.934246
1,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,370.156573,367.787571,309.379246,92.934246
2,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,370.156573,367.787571,309.379246,92.934246
3,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,370.156573,367.787571,309.379246,92.934246
4,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,2011_09_29/2011_09_29_drive_0004_sync/image_02...,370.156573,367.787571,309.379246,92.934246


In [8]:
val.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_26/2011_09_26_drive_0061_sync/image_02...,2011_09_26/2011_09_26_drive_0061_sync/image_02...,2011_09_26/2011_09_26_drive_0061_sync/image_02...,371.806866,369.427302,314.104631,88.501248
1,2011_09_26/2011_09_26_drive_0101_sync/image_02...,2011_09_26/2011_09_26_drive_0101_sync/image_02...,2011_09_26/2011_09_26_drive_0101_sync/image_02...,371.806866,369.427302,314.104631,88.501248
2,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,371.806866,369.427302,314.104631,88.501248
3,2011_10_03/2011_10_03_drive_0047_sync/image_03...,2011_10_03/2011_10_03_drive_0047_sync/image_03...,2011_10_03/2011_10_03_drive_0047_sync/image_03...,370.424992,368.054272,312.885179,94.830438
4,2011_09_30/2011_09_30_drive_0016_sync/image_03...,2011_09_30/2011_09_30_drive_0016_sync/image_03...,2011_09_30/2011_09_30_drive_0016_sync/image_03...,364.362615,362.030694,310.151266,93.752525


In [9]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [10]:
trn_camera.head()

,fx,fy,cx,cy
0,370.156573,367.787571,309.379246,92.934246
1,370.156573,367.787571,309.379246,92.934246
2,370.156573,367.787571,309.379246,92.934246
3,370.156573,367.787571,309.379246,92.934246
4,370.156573,367.787571,309.379246,92.934246


In [11]:
arch = f
bs = 4
PATH = 'Fastai_TRN'
DPATH = "KITTI/"
verbose = 500

In [12]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [13]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [14]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)

In [15]:
tfms = (trn_tfms, val_tfms)

MD1 = get_MD(trn0, val0, tfms, bs, DPATH, PATH)
MD2 = get_MD(trn1, val1, tfms, bs, DPATH, PATH)
MD3 = get_MD(trn2, val2, tfms, bs, DPATH, PATH)
MDcam = get_cam(trn_camera, val_camera, bs)

In [16]:
imgnet_mean, imgnet_std = torch.from_numpy(stats[0]).float(), torch.from_numpy(stats[1]).float()
imgnet_mean, imgnet_std = imgnet_mean.view(1,3,1,1), imgnet_std.view(1,3,1,1)
denorm = denormer(imgnet_mean, imgnet_std)

In [18]:
def train(m, opt, MD1, MD2, MD3, MDcam, denorm):
    m.train()
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []  
    
    #for i in range(2): # just for testing
    for i in range(len(MD1.trn_ds)//bs-len(losses)):
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

        d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
        #pdb.set_trace()
        loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
        # appr smooth ssim l1
        if i%verbose == 0: print(loss.data[0],
                                 details[0].data[0],
                                 details[1].data[0],
                                 details[2].data[0],
                                 details[3].data[0],
                                 sep='\t')
    return losses

In [19]:
def evaluate(m, MD1, MD2, MD3, MDcam, folder):
    
    DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
    m.eval()
    for i in range(3): 
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
        d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1, volatile=True), denorm(imgs2, volatile=True), denorm(imgs3, volatile=True) 
        #pdb.set_trace()
        d2 = d2s[0]
        # d2 = F.upsample(input=d2, scale_factor=2**di, mode='bilinear')
        cx12, cy12, dm12 = l.appr.offset.forward(trans=trans[:,0], rotation=rotation[:,0], inv_depth = d2, camera = cam)
        #pdb.set_trace()

        x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
        ivm12.volatile = True

        del dm12 
        del ivm12
        
        imgs2 = tonp(imgs2.permute(0,2,3,1))
        x12 = tonp(x12.permute(0,2,3,1))
        d2 = tonp(d2)
        
        for j in range(bs):
            save_res(imgs2[j], x12[j], d2[j][0], folder/"{}{}.png".format(i,j))

In [20]:
def fit(epoch, m, opt, MD1, MD2, MD3, MDcam):
    for i in range(epoch):
        print("--------------------epoch {} start:----------------------".format(i))
        losses = train(m, opt, MD1, MD2, MD3, MDcam)
        losses= [ str(loss) for loss in losses ]
        folder = Path("./tmp")
        folder.mkdir(exist_ok=True)
        file = folder / 'epoch{}.log'.format(i)
        with file.open('w') as f:
            f.write("\n".join(losses))
        save_model(m, str(folder / "epoch{}.M".format(i)))

In [21]:
TS = [0, 1, 0.15, 0.3, 0.5, 0.75]

In [22]:
expfolder = Path('./experiment/loss_noshuffle/')
expfolder.mkdir(parents=True, exist_ok=True)

In [23]:
for ts in TS:
    thisfolder = expfolder / "tscale_{}".format(ts)
    thisfolder.mkdir(exist_ok=True)
    
    m = TriDepth(get_resnet(), 1).cuda()
    set_trainable(m.depth.rn, False)
    l = Loss(scale=1, Tscale=ts).cuda()
    opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)
    print("training model")
    losses = train(m, opt, MD1, MD2, MD3, MDcam, denorm)
    print("saving model")
    save_model(m, thisfolder/"model.M".format(ts))
    print("writting log")
    logger = thisfolder/"log"
    losses = [ str(loss) for loss in losses]
    with logger.open('w') as f: f.write("\n".join(losses))
    print("predicting")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

training model
0.6799778342247009	0.6585083603858948	0.02146949991583824	0.6585083603858948	0.0
0.3736899495124817	0.37143850326538086	0.0022514553274959326	0.37143850326538086	0.0
0.43791699409484863	0.43410754203796387	0.0038094501942396164	0.43410754203796387	0.0
0.4624745547771454	0.45653289556503296	0.0059416452422738075	0.45653289556503296	0.0
0.4087410569190979	0.40373480319976807	0.0050062560476362705	0.40373480319976807	0.0
0.5533397793769836	0.5477355122566223	0.005604269448667765	0.5477355122566223	0.0
0.5629220604896545	0.5572003126144409	0.005721752066165209	0.5572003126144409	0.0
0.5102128982543945	0.5046278834342957	0.005585042759776115	0.5046278834342957	0.0
0.48708903789520264	0.48222237825393677	0.004866673145443201	0.48222237825393677	0.0
0.49517959356307983	0.49064552783966064	0.004534055478870869	0.49064552783966064	0.0
0.4318193197250366	0.42748332023620605	0.004335993900895119	0.42748332023620605	0.0
0.4822499454021454	0.4776625633239746	0.00458736764267087	0.477

0.42247018218040466	0.41999197006225586	0.0024782144464552402	0.39343443512916565	0.026557544246315956
0.3142019808292389	0.3122183084487915	0.0019836737774312496	0.2970315217971802	0.015186792239546776
0.3339996039867401	0.3316205143928528	0.0023790912237018347	0.3081547021865845	0.023465827107429504
0.39981764554977417	0.39731213450431824	0.002505505457520485	0.3739786446094513	0.023333493620157242
saving model
writting log
predicting
training model
0.41203346848487854	0.3856007158756256	0.026432741433382034	0.33093681931495667	0.05466390773653984
0.20414389669895172	0.20250943303108215	0.0016344698378816247	0.17187920212745667	0.030630238354206085
0.2612711191177368	0.2594374120235443	0.0018337161745876074	0.2193155139684677	0.0401218980550766
0.3270023763179779	0.3216843008995056	0.005318074952811003	0.255638062953949	0.06604624539613724
0.2833069860935211	0.27734917402267456	0.0059578195214271545	0.2040063589811325	0.07334282249212265
0.2873227894306183	0.28231480717658997	0.00500

In [25]:
for ts in TS:
    thisfolder = expfolder / "tscale_{}".format(ts)
    thisfolder.mkdir(exist_ok=True)
    m = TriDepth(get_base(), 1).cuda()
    load_model(m, thisfolder/"model.M".format(ts))
    l = Loss(scale=1, Tscale=ts).cuda()
    print("predicting")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

predicting
predicting
predicting
predicting
predicting
predicting
